In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools as it
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, precision_score, confusion_matrix
% matplotlib inline

# Vectors and Models
We've more or less settled on using TFIDF as the vectorizer of record. This is because we are looking at frequencies across a range of documents. We went to all the trouble of creating corpora of various lengths in order to have a better experimental field for the research.

Edit of 4 Apr 2017: I'm still not sure that my method is the best method possible. I'd like to try doing the same thing with pipelines to perhaps expedite the process, or at least make it grid search ready.

Twain = 1 
Wilde = 2 
Lincoln = 3
///
D_Twain = 10 
D_Wilde = 20 
D_Lincoln = 30
///
Modern = 100

1000-, 500-, and 100-record samples for each original writer - Twain, Wilde, Lincoln, and Modern

In [20]:
df_1k_H = pd.read_csv('_CSVs/df_1k_H.csv', index_col=0)
df_1k_W = pd.read_csv('_CSVs/df_1k_W.csv', index_col=0)
df_1k_S = pd.read_csv('_CSVs/df_1k_S.csv', index_col=0)

df_500_H = pd.read_csv('_CSVs/df_500_H.csv', index_col=0)
df_500_W = pd.read_csv('_CSVs/df_500_W.csv', index_col=0)
df_500_S = pd.read_csv('_CSVs/df_500_S.csv', index_col=0)

df_100_H = pd.read_csv('_CSVs/df_100_H.csv', index_col=0)
df_100_W = pd.read_csv('_CSVs/df_100_W.csv', index_col=0)
df_100_S = pd.read_csv('_CSVs/df_100_S.csv', index_col=0)

In [22]:
df_1k_H.code.value_counts()

3      1001
1      1001
100    1001
2      1001
Name: code, dtype: int64

I would like to be able so store my results of the various tests.

In [34]:
results_df = pd.DataFrame(columns=['pair', 'vec', 'features', 'model', 'm_acc', 'm_prec'])

In [35]:
results_df

Empty DataFrame
Columns: [pair, vec, features, model, m_acc, m_prec]
Index: []

In [36]:
def vec_test(vect, X, y, pair, vec):
    i = 0
    res = []
    accl = []
    precl = []
    featl = []
    while i <= 4:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3) #random_state=42)
        X_train_dtm = vect.fit_transform(X_train)
        #X_vec = vect.fit_transform(X)
        feat = X_train_dtm.shape[1]
        #print 'Features: ', feat
        featl.append(feat)
        X_test_dtm = vect.transform(X_test)
        nb = MultinomialNB()
        nb.fit(X_train_dtm, y_train)
        y_pred_class = nb.predict(X_test_dtm)
        acc = accuracy_score(y_test, y_pred_class)
        prec = precision_score(y_test, y_pred_class, average='weighted')
        accl.append(acc)
        precl.append(prec)
        #print 'Accuracy: ', acc
        #print 'Precision: ', prec
        #cm = confusion_matrix(y_test, y_pred_class)
        #cvc = pd.DataFrame(cm, columns=(nb.classes_), index=(nb.classes_))
        #print cvc
        #print
        i += 1

    mean_f = np.mean(featl)
    print "Mean Features: ", mean_f
    mean_a = np.mean(accl)
    print "Mean Accuracy: ", mean_a
    mean_p = np.mean(precl)
    print "Mean Precision: ", mean_p
    
    test_results = {}
    test_results['pair'] = pair
    test_results['vec'] = vec
    test_results['features'] = mean_f
    test_results['model'] = 'nb'
    test_results['m_acc'] = mean_a
    test_results['m_prec'] = mean_p 
    
    return test_results

#     res.append([pair, vec, mean_f, 'NB', mean_a, mean_p])
#     print res
#     r = pd.Series(res)
#     print r
#     results_df.append(r, ignore_index=True)
#     return results_df
    print
    #print cvc

In [37]:
cvec_W = CountVectorizer(decode_error='ignore') # single-word count vectorizer

In [38]:
en = vec_test(cvec_W, X_en, y_en, '1', 'cvec_W')
results_df = results_df.append(en, ignore_index=True)

Mean Features:  61078.4
Mean Accuracy:  0.967886855241
Mean Precision:  0.97095919641


In [39]:
di = vec_test(cvec_W, X_2, y_2, '2', 'cvec_W')
results_df = results_df.append(di, ignore_index=True)

Mean Features:  19704.8
Mean Accuracy:  0.872545757072
Mean Precision:  0.882568248902


In [40]:
tria  = vec_test(cvec_W, X_3, y_3, '3', 'cvec_W')
results_df = results_df.append(tria, ignore_index=True)

Mean Features:  57373.4
Mean Accuracy:  0.973300573301
Mean Precision:  0.975764118296


In [41]:
quat = vec_test(cvec_W, X_4, y_4, '4', 'cvec_W')
results_df = results_df.append(quat, ignore_index=True)

Mean Features:  60821.6
Mean Accuracy:  0.975083056478
Mean Precision:  0.976841639229


In [42]:
tst = vec_test(cvec_W, X_5, y_5, '5', 'cvec_W')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  13849.4
Mean Accuracy:  0.825913621262
Mean Precision:  0.845026753395


In [43]:
tst = vec_test(cvec_W, X_6, y_6, '6', 'cvec_W')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  49380.4
Mean Accuracy:  0.985074626866
Mean Precision:  0.985775386696


In [44]:
tst = vec_test(cvec_W, X_7, y_7, '7', 'cvec_W')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  59463.2
Mean Accuracy:  0.970491803279
Mean Precision:  0.972156808267


In [45]:
tst = vec_test(cvec_W, X_8, y_8, '8', 'cvec_W')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  5202.0
Mean Accuracy:  0.690163934426
Mean Precision:  0.765488503083


In [46]:
tst = vec_test(cvec_W, X_9, y_9, '9', 'cvec_W') # mean precision of .9155
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  25255.0
Mean Accuracy:  0.95737704918
Mean Precision:  0.9632463207


In [47]:
cvec_WS = CountVectorizer(decode_error='ignore', stop_words='english') # 

In [48]:
tst = vec_test(cvec_WS, X_en, y_en, '1', 'cvec_WS')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  60884.6
Mean Accuracy:  0.975873544093
Mean Precision:  0.977644617193


In [49]:
tst = vec_test(cvec_WS, X_2, y_2, '2', 'cvec_WS') # .91 accuracy score
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  19450.2
Mean Accuracy:  0.888851913478
Mean Precision:  0.893664009969


In [50]:
tst = vec_test(cvec_WS, X_3, y_3, '3', 'cvec_WS')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  56938.6
Mean Accuracy:  0.978705978706
Mean Precision:  0.980412940909


In [51]:
tst = vec_test(cvec_WS, X_4, y_4, '4', 'cvec_WS')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  60297.2
Mean Accuracy:  0.978073089701
Mean Precision:  0.979676001636


In [52]:
tst = vec_test(cvec_WS, X_5, y_5, '5', 'cvec_WS')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  13574.8
Mean Accuracy:  0.864451827243
Mean Precision:  0.872063303234


In [53]:
tst = vec_test(cvec_WS, X_6, y_6, '6', 'cvec_WS')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  49196.8
Mean Accuracy:  0.9864013267
Mean Precision:  0.986975741288


In [54]:
tst = vec_test(cvec_WS, X_7, y_7, '7', 'cvec_WS')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  59470.6
Mean Accuracy:  0.981967213115
Mean Precision:  0.982878398394


In [55]:
tst = vec_test(cvec_WS, X_8, y_8, '8', 'cvec_WS')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  4940.8
Mean Accuracy:  0.711475409836
Mean Precision:  0.722852795971


In [56]:
tst = vec_test(cvec_WS, X_9, y_9, '9', 'cvec_WS') # .90 accuracy score
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  25135.8
Mean Accuracy:  0.993442622951
Mean Precision:  0.993581983968


In [57]:
cvec_WS2 = CountVectorizer(decode_error='ignore', stop_words='english', ngram_range=(2,2))

In [58]:
tst = vec_test(cvec_WS2, X_en, y_en, '1', 'cvec_WS2')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  993194.8
Mean Accuracy:  0.952579034942
Mean Precision:  0.959488643417


In [59]:
tst = vec_test(cvec_WS2, X_2, y_2, '2', 'cvec_WS2')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  90195.4
Mean Accuracy:  0.670216306156
Mean Precision:  0.682999393047


In [60]:
tst = vec_test(cvec_WS2, X_3, y_3, '3', 'cvec_WS2')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  846541.2
Mean Accuracy:  0.973300573301
Mean Precision:  0.975594848946


In [61]:
tst = vec_test(cvec_WS2, X_4, y_4, '4', 'cvec_WS2')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  996836.0
Mean Accuracy:  0.969435215947
Mean Precision:  0.972620313444


In [62]:
tst = vec_test(cvec_WS2, X_5, y_5, '5', 'cvec_WS2')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  46394.8
Mean Accuracy:  0.5926910299
Mean Precision:  0.637532671411


In [63]:
tst = vec_test(cvec_WS2, X_6, y_6, '6', 'cvec_WS2')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  603898.6
Mean Accuracy:  0.991708126036
Mean Precision:  0.99189980331


In [64]:
tst = vec_test(cvec_WS2, X_7, y_7, '7', 'cvec_WS2')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  987591.2
Mean Accuracy:  0.970491803279
Mean Precision:  0.973417632947


In [65]:
tst = vec_test(cvec_WS2, X_8, y_8, '8', 'cvec_WS2')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  9640.2
Mean Accuracy:  0.43606557377
Mean Precision:  0.657457745959


In [66]:
tst = vec_test(cvec_WS2, X_9, y_9, '9', 'cvec_WS2') # mean precision of .95!
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  178379.4
Mean Accuracy:  0.95737704918
Mean Precision:  0.959501037431


In [67]:
cvec_NS4 = CountVectorizer(decode_error='ignore', stop_words='english', analyzer='char', ngram_range=(1,4))

In [68]:
tst = vec_test(cvec_NS4, X_en, y_en, '1', 'cvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  165937.0
Mean Accuracy:  0.964392678869
Mean Precision:  0.967504644931


In [69]:
tst = vec_test(cvec_NS4, X_2, y_2, '2', 'cvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  41793.6
Mean Accuracy:  0.891014975042
Mean Precision:  0.89719450908


In [70]:
tst = vec_test(cvec_NS4, X_3, y_3, '3', 'cvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  155432.2
Mean Accuracy:  0.972153972154
Mean Precision:  0.974427249114


In [71]:
tst = vec_test(cvec_NS4, X_4, y_4, '4', 'cvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  164772.0
Mean Accuracy:  0.967109634551
Mean Precision:  0.969670647744


In [72]:
tst = vec_test(cvec_NS4, X_5, y_5, '5', 'cvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  33578.8
Mean Accuracy:  0.876744186047
Mean Precision:  0.886696214853


In [73]:
tst = vec_test(cvec_NS4, X_6, y_6, '6', 'cvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  137764.2
Mean Accuracy:  0.974129353234
Mean Precision:  0.97650048586


In [74]:
tst = vec_test(cvec_NS4, X_7, y_7, '7', 'cvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  163710.0
Mean Accuracy:  0.970491803279
Mean Precision:  0.973142593363


In [75]:
tst = vec_test(cvec_NS4, X_8, y_8, '8', 'cvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  19232.4
Mean Accuracy:  0.83606557377
Mean Precision:  0.853945986391


In [76]:
tst = vec_test(cvec_NS4, X_9, y_9, '9', 'cvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  80756.2
Mean Accuracy:  0.96393442623
Mean Precision:  0.967746100902


In [77]:
def tvec_test(vect, X, y, pair, vec):
    i = 0
    res = []
    accl = []
    precl = []
    featl = []
    while i <= 4:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.3) #random_state=42)
        X_train_dtm = vect.fit_transform(X_train)
        #X_vec = vect.fit_transform(X)
        feat = X_train_dtm.shape[1]
        #print 'Features: ', feat
        featl.append(feat)
        X_test_dtm = vect.transform(X_test)
        nb = MultinomialNB()
        nb.fit(X_train_dtm, y_train)
        y_pred_class = nb.predict(X_test_dtm)
        acc = accuracy_score(y_test, y_pred_class)
        prec = precision_score(y_test, y_pred_class, average='weighted')
        accl.append(acc)
        precl.append(prec)
        #print 'Accuracy: ', acc
        #print 'Precision: ', prec
        #cm = confusion_matrix(y_test, y_pred_class)
        #cvc = pd.DataFrame(cm, columns=(nb.classes_), index=(nb.classes_))
        #print cvc
        #print
        i += 1
    mean_f = np.mean(featl)
    print "Mean Features: ", mean_f
    mean_a = np.mean(accl)
    print "Mean Accuracy: ", mean_a
    mean_p = np.mean(precl)
    print "Mean Precision: ", mean_p
    res.append([pair, vec, mean_f, 'NB', mean_a, mean_p])
    results_df.append(res)
    
    test_results = {}
    test_results['pair'] = pair
    test_results['vec'] = vec
    test_results['features'] = mean_f
    test_results['model'] = 'nb'
    test_results['m_acc'] = mean_a
    test_results['m_prec'] = mean_p 
    
    return test_results
    
    print
    #print cvc

In [78]:
tvect = TfidfVectorizer()

In [79]:
tst = tvec_test(tvect, X_en, y_en, '1', 'tvect')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  61233.2
Mean Accuracy:  0.795840266223
Mean Precision:  0.887555206189


In [80]:
tst = tvec_test(tvect, X_2, y_2, '2', 'tvect')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  19761.4
Mean Accuracy:  0.822961730449
Mean Precision:  0.846229121216


In [81]:
tst = tvec_test(tvect, X_3, y_3, '1', 'tvect')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  57338.2
Mean Accuracy:  0.841932841933
Mean Precision:  0.904235375629


In [82]:
tst = tvec_test(tvect, X_4, y_4, '4', 'tvect')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  60566.0
Mean Accuracy:  0.874750830565
Mean Precision:  0.919283984586


In [83]:
tst = tvec_test(tvect, X_5, y_5, '5', 'tvect')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  13842.2
Mean Accuracy:  0.781063122924
Mean Precision:  0.822702337139


In [84]:
tst = tvec_test(tvect, X_6, y_6, '6', 'tvect')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  49537.4
Mean Accuracy:  0.89320066335
Mean Precision:  0.925693046872


In [85]:
tst = tvec_test(tvect, X_7, y_7, '7', 'tvect')
results_df = results_df.append(tst, ignore_index=True)

/Users/charlesrice/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1074: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Mean Features:  60154.0
Mean Accuracy:  0.696721311475
Mean Precision:  0.809769192906


In [86]:
tst = tvec_test(tvect, X_8, y_8, '8', 'tvect')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  5241.2
Mean Accuracy:  0.56393442623
Mean Precision:  0.742230756905


In [87]:
tst = tvec_test(tvect, X_9, y_9, '9', 'tvect')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  25231.0
Mean Accuracy:  0.737704918033
Mean Precision:  0.834474707337


In [88]:
tvec_NS4 = TfidfVectorizer(analyzer='char', stop_words='english', ngram_range=(1, 4))

In [89]:
tst = tvec_test(tvec_NS4, X_en, y_en, '1', 'tvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  166026.6
Mean Accuracy:  0.916472545757
Mean Precision:  0.929046470154


In [91]:
tst = tvec_test(tvec_NS4, X_2, y_2, '2', 'tvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  41907.4
Mean Accuracy:  0.807820299501
Mean Precision:  0.838572011341


In [92]:
tst = tvec_test(tvec_NS4, X_3, y_3, '3', 'tvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  155453.4
Mean Accuracy:  0.852088452088
Mean Precision:  0.884870668889


In [93]:
tst = tvec_test(tvec_NS4, X_4, y_4, '4', 'tvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  165584.6
Mean Accuracy:  0.77707641196
Mean Precision:  0.863813923356


In [94]:
tst = tvec_test(tvec_NS4, X_5, y_5, '5', 'tvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  33451.6
Mean Accuracy:  0.740863787375
Mean Precision:  0.816188402398


In [95]:
tst = tvec_test(tvec_NS4, X_6, y_6, '6', 'tvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  138119.8
Mean Accuracy:  0.866998341625
Mean Precision:  0.904544667539


In [96]:
tst = tvec_test(tvec_NS4, X_7, y_7, '7', 'tvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  164966.2
Mean Accuracy:  0.434426229508
Mean Precision:  0.454604735197


In [97]:
tst = tvec_test(tvec_NS4, X_8, y_8, '8', 'tvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  19109.6
Mean Accuracy:  0.54262295082
Mean Precision:  0.758731175658


In [98]:
tst = tvec_test(tvec_NS4, X_9, y_9, '9', 'tvec_NS4')
results_df = results_df.append(tst, ignore_index=True)

Mean Features:  80503.2
Mean Accuracy:  0.634426229508
Mean Precision:  0.687544043503


In [112]:
results_df.describe()

features      m_acc     m_prec
count      54.000000  54.000000  54.000000
mean   143253.566667   0.853479   0.885833
std    249304.322470   0.147355   0.113428
min      4940.800000   0.434426   0.454605
25%     25237.000000   0.784757   0.840186
50%     59466.900000   0.889933   0.911914
75%    151104.100000   0.970492   0.973349
max    996836.000000   0.993443   0.993582

# A Wild Data Approaches!

Having played around with features and variables for most of a day, I'm confident in the naive bayes model, using the 4 character inclusive ngram count vectorizer as the model for success. Although there is further testing to be done on the extant corpora, and refinements that can be made, I would prefer to get it over and done with.

Details of the winning model: CVEC NS4, pair 4

In [113]:
twain_w = pd.read_csv("../code/test_quotes/twain_quotes.csv", encoding="utf-8")

twain_w.code.fillna(value='??', inplace=True)

In [114]:
wilde_quotes = pd.read_csv('../code/test_quotes/wilde_quotes.csv', encoding='utf-8')

wilde_quotes.code.fillna(value='??', inplace=True)

In [115]:
lincoln_quotes = pd.read_csv('../code/test_quotes/lincoln_quotes.csv', encoding='utf-8')

lincoln_quotes.code.fillna(value='??', inplace=True)

lincoln_quotes.head()

quote code
0  The best way to predict the future is to creat...   30
1  I am for those means which will give the great...    3
2  Having thus chosen our course without guile an...    3
3  You have to do your own growing no matter how ...   ??
4  I have been driven many times to my knees by t...   ??

In [221]:
frames = [twain_w, wilde_quotes, lincoln_quotes]

wild_data = pd.concat(frames, ignore_index=True)

In [ ]:
wild_data.to_csv('quotes.csv', index=False, encoding='utf-8')

In [ ]:
wild_data = pd.read_csv('quotes.csv')

In [222]:
wild_data # All good!

quote code
0     If you pick up a starving dog and make him pro...    1
1     Sing like no one's listening, love like you've...   10
2     But when the time comes that a man has had his...   ??
3     No real estate is permanently valuable but the...   ??
4     More than once I had seen a noble who had gott...   ??
5     The first half of my life I went to school, th...   ??
6     Some people get an education without going to ...   ??
7     Between believing a thing and thinking you kno...   ??
8     The first of April is the day we remember what...   ??
9     Keep away from those who try to belittle your ...   ??
10    Courage is resistance to fear, mastery of fear...   ??
11    Don't go around saying the world owes you a li...   ??
12    Forgiveness is the fragrance that the violet s...   ??
13    Anger is an acid that can do more harm to the ...   ??
14    Persons attempting to find a motive in this na...   ??
15    Each person is born to one possession which ou...   ??
16    The gentle reader will never, never know what ...   ??
17    When I'm playful I use the meridians of longit...   ??
18    Delicacy - a sad, sad false delicacy - robs li...   ??
19    Well, we can take you to meet Napoleon -- but ...   ??
20    There has been only one Christian. They caught...    1
21    By his father he is English, by his mother he ...   ??
22    The perfection of wisdom, and the end of true ...    1
23    No one is willing to acknowledge a fault in hi...   10
24    Part of my plan has been to try to pleasantly ...    1
25    I think I can say, and say with pride, that we...   ??
26    If there was two birds sitting on a fence, he ...   ??
27    I saw a cat yesterday with 4 legs and yet it w...   ??
28    There are two times in a man's life when he sh...   ??
29    God made the Idiot for practice, and then He m...   ??
...                                                 ...  ...
2498  Labor is prior to and independent of capital. ...   30
2499  He bores me. He ought to have stuck to his fly...   30
2500  You cannot help men permanently by doing for t...   30
2501  America will never be destroyed from the outsi...   30
2502  Now  I say to you  my fellow-citizens  that in...   30
2503  As a result of the war  corporations have been...   30
2504  The money powers prey upon the nation in times...   30
2505  I like to see a man proud of the place in whic...   30
2506  Any nation that does not honor its heroes will...   30
2507  There is no room for two distinct races of whi...   30
2508  Congressmen who willfully take actions during ...   30
2509  If you look for the bad in mankind expecting t...   30
2510  Money is the creature of law and creation of t...   30
2511  If this is coffee  please bring me some tea; b...   30
2512  I will study and get ready  and perhaps my cha...   30
2513  I am not concerned that you fall; I am concern...   30
2514  I destroy my enemies when I make them my friends.   30
2515  If I had another face  do you think I would we...   30
2516  It is better to be silent and be thought a foo...   30
2517  Marriage is neither heaven nor hell  it is sim...   30
2518  The best way to get a bad law repealed is to e...   30
2519  And in the end  it's not the years in your lif...   30
2520  I am a firm believer in the people. If given t...   30
2521  The people will save their government  if the ...   30
2522  You can fool some of the people all of the tim...   30
2523  When I left Springfield I asked the people to ...   30
2524  We  on our side  are praying Him to give us vi...   30
2525  It's my experience that folks who have no vice...   30
2526  What is to be  will be  and no prayers of ours...   30
2527                      Let's bomb so we can go home.   30

[2528 rows x 2 columns]

In [223]:
cvec_NS4 = CountVectorizer(decode_error='ignore', stop_words='english', analyzer='char', ngram_range=(1,4))

In [224]:
w_nb = MultinomialNB()

w_cvec = cvec_NS4

In [225]:
X_c_dtm = w_cvec.fit_transform(X_4)

In [226]:
X_w = wild_data['quote'].values

In [227]:
X_w_dtm = w_cvec.transform(X_w)

In [228]:
trained_nb = w_nb.fit(X_c_dtm, y_4)

In [229]:
predictions_1 = w_nb.predict(X_w_dtm)

In [ ]:
predictions_1

In [ ]:
probs = w_nb.predict_proba(X_w_dtm)

In [ ]:
probs

In [230]:
wild_data['Predictions_1'] = predictions_1

In [231]:
wild_data.Predictions_1.value_counts()

1      938
2      879
3      605
100    106
Name: Predictions_1, dtype: int64

In [ ]:
wild_data[wild_data.code==30]

In [ ]:
TN_L = float(len(wild_data[wild_data.code==30]))

In [ ]:
FP_L = float(len(wild_data[(wild_data.code == 30) & (wild_data.Prediction == 3.0)]))

In [ ]:
FP_L/TN_L

So just over 50 percent of the falseley attributed quotations were predicted to be Lincoln by the model, including the quotation that started this whole thing.

In [ ]:
TN_W = float(len(wild_data[wild_data.code==20]))
FP_W = float(len(wild_data[(wild_data.code==20) & (wild_data.Prediction==2.0)]))

In [ ]:
FP_W/TN_W

Wilde was a little better, just under 40% of the quotes falsely attributed to Wilde were predicted to be his according to the model.

In [ ]:
TN_T = float(len(wild_data[wild_data.code==10]))
FP_T = float(len(wild_data[(wild_data.code==10) & (wild_data.Prediction==1.0)]))

In [ ]:
FP_T/TN_T

In [ ]:
wild_data[wild_data.code == wild_data.Prediction]

In [ ]:
wild_data[(wild_data.code != '??') & (wild_data.Prediction == wild_data.code)]

And of the 91 quotations with known attributions, the model correctly guessed 9, about 10%.

Those results were disappointing, to say the last, so I decided to correct the class imbalance.

# Slightly Less Wild Data Approaches

In [232]:
cvec_WS2 = CountVectorizer(decode_error='ignore', stop_words='english', ngram_range=(2,2))

In [ ]:
cvec_WS2, X_6, y_6

In [233]:
w_nb_2 = MultinomialNB()

w_cvec_2 = cvec_WS2

In [234]:
X_c_dtm2 = w_cvec_2.fit_transform(X_6)

In [235]:
X_w2 = wild_data['quote'].values

In [236]:
X_w_dtm2 = w_cvec_2.transform(X_w2)

In [237]:
trained_nb = w_nb_2.fit(X_c_dtm2, y_6)

In [238]:
predictions_2 = w_nb_2.predict(X_w_dtm2)

In [127]:
predictions_2

array([  1, 100,   1, ...,   3,   3, 100])

In [239]:
wild_data['Predictions_2'] = predictions_2

In [240]:
wild_data

quote code  Predictions_1  \
0     If you pick up a starving dog and make him pro...    1              1   
1     Sing like no one's listening, love like you've...   10              1   
2     But when the time comes that a man has had his...   ??              1   
3     No real estate is permanently valuable but the...   ??              3   
4     More than once I had seen a noble who had gott...   ??              1   
5     The first half of my life I went to school, th...   ??              2   
6     Some people get an education without going to ...   ??              1   
7     Between believing a thing and thinking you kno...   ??            100   
8     The first of April is the day we remember what...   ??              3   
9     Keep away from those who try to belittle your ...   ??            100   
10    Courage is resistance to fear, mastery of fear...   ??              3   
11    Don't go around saying the world owes you a li...   ??              1   
12    Forgiveness is the fragrance that the violet s...   ??              2   
13    Anger is an acid that can do more harm to the ...   ??              3   
14    Persons attempting to find a motive in this na...   ??              3   
15    Each person is born to one possession which ou...   ??              1   
16    The gentle reader will never, never know what ...   ??              3   
17    When I'm playful I use the meridians of longit...   ??              1   
18    Delicacy - a sad, sad false delicacy - robs li...   ??            100   
19    Well, we can take you to meet Napoleon -- but ...   ??              1   
20    There has been only one Christian. They caught...    1              2   
21    By his father he is English, by his mother he ...   ??              1   
22    The perfection of wisdom, and the end of true ...    1              3   
23    No one is willing to acknowledge a fault in hi...   10              3   
24    Part of my plan has been to try to pleasantly ...    1              1   
25    I think I can say, and say with pride, that we...   ??              3   
26    If there was two birds sitting on a fence, he ...   ??              1   
27    I saw a cat yesterday with 4 legs and yet it w...   ??              1   
28    There are two times in a man's life when he sh...   ??              1   
29    God made the Idiot for practice, and then He m...   ??              1   
...                                                 ...  ...            ...   
2498  Labor is prior to and independent of capital. ...   30              3   
2499  He bores me. He ought to have stuck to his fly...   30              1   
2500  You cannot help men permanently by doing for t...   30              3   
2501  America will never be destroyed from the outsi...   30              3   
2502  Now  I say to you  my fellow-citizens  that in...   30              3   
2503  As a result of the war  corporations have been...   30              3   
2504  The money powers prey upon the nation in times...   30              3   
2505  I like to see a man proud of the place in whic...   30              2   
2506  Any nation that does not honor its heroes will...   30              3   
2507  There is no room for two distinct races of whi...   30              3   
2508  Congressmen who willfully take actions during ...   30              3   
2509  If you look for the bad in mankind expecting t...   30            100   
2510  Money is the creature of law and creation of t...   30              3   
2511  If this is coffee  please bring me some tea; b...   30              1   
2512  I will study and get ready  and perhaps my cha...   30              1   
2513  I am not concerned that you fall; I am concern...   30              2   
2514  I destroy my enemies when I make them my friends.   30              3   
2515  If I had another face  do you think I would we...   30              1   
2516  It is better to be silent and be thought a foo...   30              1   
2517  Marriage is nei

In [130]:
wild_data[wild_data.code==30]

quote code  Predictions
2085  The best way to predict the future is to creat...   30          100
2093  And in the end it's not the years in your life...   30            3
2497  Nearly all men can stand adversity but if you ...   30            2
2498  Labor is prior to and independent of capital. ...   30            3
2499  He bores me. He ought to have stuck to his fly...   30          100
2500  You cannot help men permanently by doing for t...   30            3
2501  America will never be destroyed from the outsi...   30            3
2502  Now  I say to you  my fellow-citizens  that in...   30            3
2503  As a result of the war  corporations have been...   30            1
2504  The money powers prey upon the nation in times...   30            3
2505  I like to see a man proud of the place in whic...   30            1
2506  Any nation that does not honor its heroes will...   30            1
2507  There is no room for two distinct races of whi...   30            3
2508  Congressmen who willfully take actions during ...   30            3
2509  If you look for the bad in mankind expecting t...   30            3
2510  Money is the creature of law and creation of t...   30          100
2511  If this is coffee  please bring me some tea; b...   30            2
2512  I will study and get ready  and perhaps my cha...   30            1
2513  I am not concerned that you fall; I am concern...   30            3
2514  I destroy my enemies when I make them my friends.   30            2
2515  If I had another face  do you think I would we...   30            2
2516  It is better to be silent and be thought a foo...   30            1
2517  Marriage is neither heaven nor hell  it is sim...   30            2
2518  The best way to get a bad law repealed is to e...   30            1
2519  And in the end  it's not the years in your lif...   30            3
2520  I am a firm believer in the people. If given t...   30            3
2521  The people will save their government  if the ...   30            3
2522  You can fool some of the people all of the tim...   30          100
2523  When I left Springfield I asked the people to ...   30            3
2524  We  on our side  are praying Him to give us vi...   30            3
2525  It's my experience that folks who have no vice...   30            3
2526  What is to be  will be  and no prayers of ours...   30            3
2527                      Let's bomb so we can go home.   30          100

In [131]:
wild_data[wild_data.Predictions==100]

quote code  Predictions
1     Sing like no one's listening, love like you've...   10          100
3     No real estate is permanently valuable but the...   ??          100
7     Between believing a thing and thinking you kno...   ??          100
9     Keep away from those who try to belittle your ...   ??          100
11    Don't go around saying the world owes you a li...   ??          100
24    Part of my plan has been to try to pleasantly ...    1          100
37    The time to begin writing an article is when y...   ??          100
45    When it comes down to pure ornamental cursing,...   ??          100
63    It's a great place to live, but I wouldn't wan...   ??          100
82    He said, 'I've never seen a festival where the...   ??          100
83    There's always a hole in theories somewhere if...   ??          100
96    Part of the secret of success in life is to ea...   ??          100
98    No matter how healthy a man's morals may be wh...   ??          100
102   Thunder is impressive, but it is lightning tha...   ??          100
108   Ours is the land of the free nobody denies tha...   ??          100
116   As a rule we develop a borrowed European idea ...   ??          100
117   Keep away from small people who try to belittl...   ??          100
129   The self taught man seldom knows anything accu...   ??          100
142   I owe a friend a dozen chickens, and I believe...   ??          100
150   None but the dead have free speech. None but t...   ??          100
174   Education does not mean teaching people to kno...   ??          100
201   Concerning the difference between man and the ...   ??          100
212   It's better to keep quiet and have people thin...   ??          100
221   Keep away from people who try to belittle your...   ??          100
238   Everybody's private motto: It's better to be p...   ??          100
241   A genuine expert can always foretell a thing t...   ??          100
243   None of us can have as many virtues as the fou...   ??          100
244   Mardi Gras is a thing which could hardly exist...   ??          100
259   They spell it Vinci and pronounce it Vinchy; f...   ??          100
274   Well enough for old folks to rise early, becau...   ??          100
...                                                 ...  ...          ...
2180  The banner signified the successful completion...   ??          100
2181  [Bush also defended the] Mission Accomplished ...   ??          100
2182  [Bush also defended the] Mission Accomplished ...   ??          100
2185  Whatever spiteful fools may say Each jealous r...   ??          100
2199  Whenever there is a conflict between human rig...   ??          100
2217  Sir my concern is not whether God is on our si...   ??          100
2218  How many legs does a dog have if you call his ...   ??          100
2224  If I am killed I can die but once; but to live...   ??          100
2250  For people who like that kind of a book that i...   ??          100
2272  Force is all-conquering but its victories are ...   ??          100
2282  This human struggle and scramble for office fo...   ??          100
2293  Tell me what brand of whiskey that Grant drink...   ??          100
2305  We know nothing of what will happen in future ...   ??          100
2326  If I care to listen to every criticism let alo...   ??          100
2365  If I had eight hours to chop down a tree I'd s...   ??          100
2366  You can fool all the people some of the time a...   ??          100
2387  The things I want to know are in books; my bes...   ??          100
2389  How many legs does a dog have if you call the ...   ??          100
2402  Sir my concern is not whether God is on our si...   ??          100
2414  Things may come to those who wait but only the...   ??          100
2415   I will prepare and some day my chance will come.   ??          100
2420  Don't worry when you are not recognized but st...   ??          100
2426  When I am getting ready to reason with a man I..

In [135]:
wild_data[(wild_data.code != '??') & (wild_data.Predictions == 100)]

quote code  Predictions
1     Sing like no one's listening, love like you've...   10          100
24    Part of my plan has been to try to pleasantly ...    1          100
1155  I didn't attend the funeral, but I sent a nice...   10          100
1164  The two most important days in your life are t...   10          100
1165         The lack of money is the root of all evil.   10          100
1167  He who fights with monsters should look to it ...   10          100
1173  There are three kinds of lies: lies damned lie...   10          100
1178  Whiskey is for drinking. Water is for fighting...   10          100
1180  Everybody talks about the weather, but nobody ...   10          100
1183  Censorship is telling a man he can't have a st...   10          100
1186  Don't believe the world owes you a living. The...   10          100
2081  I don't want to go to heaven. None of my frien...   20          100
2083  If you want to tell people the truth, make the...   20          100
2085  The best way to predict the future is to creat...   30          100
2499  He bores me. He ought to have stuck to his fly...   30          100
2510  Money is the creature of law and creation of t...   30          100
2522  You can fool some of the people all of the tim...   30          100
2527                      Let's bomb so we can go home.   30          100

In [134]:
w_cvec_2.get_feature_names()

[u'00 00',
 u'00 000',
 u'00 100',
 u'00 12',
 u'00 1814',
 u'00 22',
 u'00 35',
 u'00 350',
 u'00 86',
 u'00 barrel',
 u'00 barrels',
 u'00 bath',
 u'00 bernie',
 u'00 cash',
 u'00 choice',
 u'00 chores',
 u'00 class',
 u'00 crockery',
 u'00 crops',
 u'00 dining',
 u'00 dozen',
 u'00 dry',
 u'00 friday',
 u'00 furs',
 u'00 haste',
 u'00 houses',
 u'00 included',
 u'00 know',
 u'00 left',
 u'00 lunch',
 u'00 pm',
 u'00 previous',
 u'00 railroad',
 u'00 rendering',
 u'00 saturday',
 u'00 smith',
 u'00 total',
 u'00 victorian',
 u'000 00',
 u'000 000',
 u'000 100',
 u'000 107',
 u'000 12',
 u'000 120',
 u'000 15',
 u'000 17',
 u'000 23d',
 u'000 26',
 u'000 50',
 u'000 60',
 u'000 abraham',
 u'000 accept',
 u'000 accidentally',
 u'000 accounted',
 u'000 acres',
 u'000 added',
 u'000 additional',
 u'000 advanced',
 u'000 afford',
 u'000 allies',
 u'000 annually',
 u'000 apiece',
 u'000 appropriation',
 u'000 army',
 u'000 arsenal',
 u'000 asking',
 u'000 australasian',
 u'000 average',
 u

# Once More With Feeling!

In [241]:
cvec_NS4 = CountVectorizer(decode_error='ignore', stop_words='english', analyzer='char', ngram_range=(1,4))

In [242]:
cvec_w_3 = cvec_NS4

In [243]:
w_nb_3 = MultinomialNB()

X_c_dtm3 = cvec_w_3.fit_transform(X_en)

In [244]:
X_w3 = wild_data['quote'].values

In [140]:
X_w_dtm3 = cvec_w_3.transform(X_w3)

In [141]:
trained_nb = w_nb_3.fit(X_c_dtm3, y_en)

In [142]:
predictions_3 = w_nb_3.predict(X_w_dtm3)

In [143]:
wild_data['Predictions_3'] = predictions_3

In [213]:
wild_data.head()

quote code  Predictions  \
0  If you pick up a starving dog and make him pro...    1            1   
1  Sing like no one's listening, love like you've...   10          100   
2  But when the time comes that a man has had his...   ??            1   
3  No real estate is permanently valuable but the...   ??          100   
4  More than once I had seen a noble who had gott...   ??            1   

   Predictions 3  
0              1  
1              1  
2              1  
3              3  
4              1

In [145]:
wild_data[(wild_data.code != '??')]

quote code  Predictions  \
0     If you pick up a starving dog and make him pro...    1            1   
1     Sing like no one's listening, love like you've...   10          100   
20    There has been only one Christian. They caught...    1            3   
22    The perfection of wisdom, and the end of true ...    1            1   
23    No one is willing to acknowledge a fault in hi...   10            3   
24    Part of my plan has been to try to pleasantly ...    1          100   
684                Sacred cows make the best hamburger.   10            1   
1154  In the first place, God made idiots. That was ...    1            1   
1155  I didn't attend the funeral, but I sent a nice...   10          100   
1161  I take my only exercise acting as pallbearer a...   10            3   
1162  I don't give a damn for a man who can only spe...   10            1   
1163  Kindness is the language which the deaf can he...   10            3   
1164  The two most important days in your life are t...   10          100   
1165         The lack of money is the root of all evil.   10          100   
1166  If you don't read the newspaper, you are uninf...   10            1   
1167  He who fights with monsters should look to it ...   10          100   
1168  Be careful about reading health books. You may...   10            3   
1169  When a child turns 12, he should be kept in a ...   10            1   
1170  Describing her first day back in grade school ...   10            1   
1171           Wagner's music is better than it sounds.   10            2   
1172  The minority is always in the right. The major...   10            3   
1173  There are three kinds of lies: lies damned lie...   10          100   
1174  Wagner's music is better thanscribing her firs...   10            2   
1175  The coldest winter I ever spent was a summer i...   10            1   
1176                        Golf is a good walk spoiled   10            3   
1177  I would have written a shorter letter, but I d...   10            1   
1178  Whiskey is for drinking. Water is for fighting...   10          100   
1179  A banker is a fellow who lends you his umbrell...   10            3   
1180  Everybody talks about the weather, but nobody ...   10          100   
1181  Twenty years from now you will be more disappo...   10            1   
...                                                 ...  ...          ...   
2498  Labor is prior to and independent of capital. ...   30            3   
2499  He bores me. He ought to have stuck to his fly...   30          100   
2500  You cannot help men permanently by doing for t...   30            3   
2501  America will never be destroyed from the outsi...   30            3   
2502  Now  I say to you  my fellow-citizens  that in...   30            3   
2503  As a result of the war  corporations have been...   30            1   
2504  The money powers prey upon the nation in times...   30            3   
2505  I like to see a man proud of the place in whic...   30            1   
2506  Any nation that does not honor its heroes will...   30            1   
2507  There is no room for two distinct races of whi...   30            3   
2508  Congressmen who willfully take actions during ...   30            3   
2509  If you look for the bad in mankind expecting t...   30            3   
2510  Money is the creature of law and creation of t...   30          100   
2511  If this is coffee  please bring me some tea; b...   30            2   
2512  I will study and get ready  and perhaps my cha...   30            1   
2513  I am not concerned that you fall; I am concern...   30            3   
2514  I destroy my enemies when I make them my friends.   30            2   
2515  If I had another face  do you think I would we...   30            2   
2516  It is better to be silent and be thought a foo...   30            1   
2517  Marriage is neither heaven nor hell  it is sim...   30            2   
2518  The best way to get a bad law repealed i

In [146]:
wild_data[(wild_data.code == wild_data.Predictions)]

quote code  Predictions  \
0     If you pick up a starving dog and make him pro...    1            1   
22    The perfection of wisdom, and the end of true ...    1            1   
1154  In the first place, God made idiots. That was ...    1            1   
1189  one should never copy the method of an artist....    2            2   
1191  The condition of perfection is idleness: the a...    2            2   
1986  It is only shallow people who do not judge by ...    2            2   
1990  A woman will flirt with anybody in the world a...    2            2   
1996  To get into the best society nowadays one has ...    2            2   
1997  It is very vulgar to talk about one's business...    2            2   
2086  I am for those means which will give the great...    3            3   
2087  Having thus chosen our course without guile an...    3            3   

      Predictions 3  
0                 1  
22                3  
1154              1  
1189              2  
1191              3  
1986              3  
1990              2  
1996              2  
1997              2  
2086              3  
2087              1

In [190]:
k_preds = pd.DataFrame(wild_data[wild_data.code != '??'])

In [191]:
k_preds = k_preds.reset_index()

In [206]:
k_preds

quote  code  Predictions  \
0   If you pick up a starving dog and make him pro...     1            1   
1   Sing like no one's listening, love like you've...   100          100   
2   There has been only one Christian. They caught...     1            3   
3   The perfection of wisdom, and the end of true ...     1            1   
4   No one is willing to acknowledge a fault in hi...   100            3   
5   Part of my plan has been to try to pleasantly ...     1          100   
6                Sacred cows make the best hamburger.   100            1   
7   In the first place, God made idiots. That was ...     1            1   
8   I didn't attend the funeral, but I sent a nice...   100          100   
9   I take my only exercise acting as pallbearer a...   100            3   
10  I don't give a damn for a man who can only spe...   100            1   
11  Kindness is the language which the deaf can he...   100            3   
12  The two most important days in your life are t...   100          100   
13         The lack of money is the root of all evil.   100          100   
14  If you don't read the newspaper, you are uninf...   100            1   
15  He who fights with monsters should look to it ...   100          100   
16  Be careful about reading health books. You may...   100            3   
17  When a child turns 12, he should be kept in a ...   100            1   
18  Describing her first day back in grade school ...   100            1   
19           Wagner's music is better than it sounds.   100            2   
20  The minority is always in the right. The major...   100            3   
21  There are three kinds of lies: lies damned lie...   100          100   
22  Wagner's music is better thanscribing her firs...   100            2   
23  The coldest winter I ever spent was a summer i...   100            1   
24                        Golf is a good walk spoiled   100            3   
25  I would have written a shorter letter, but I d...   100            1   
26  Whiskey is for drinking. Water is for fighting...   100          100   
27  A banker is a fellow who lends you his umbrell...   100            3   
28  Everybody talks about the weather, but nobody ...   100          100   
29  Twenty years from now you will be more disappo...   100            1   
..                                                ...   ...          ...   
61  Labor is prior to and independent of capital. ...   100            3   
62  He bores me. He ought to have stuck to his fly...   100          100   
63  You cannot help men permanently by doing for t...   100            3   
64  America will never be destroyed from the outsi...   100            3   
65  Now  I say to you  my fellow-citizens  that in...   100            3   
66  As a result of the war  corporations have been...   100            1   
67  The money powers prey upon the nation in times...   100            3   
68  I like to see a man proud of the place in whic...   100            1   
69  Any nation that does not honor its heroes will...   100            1   
70  There is no room for two distinct races of whi...   100            3   
71  Congressmen who willfully take actions during ...   100            3   
72  If you look for the bad in mankind expecting t...   100            3   
73  Money is the creature of law and creation of t...   100          100   
74  If this is coffee  please bring me some tea; b...   100            2   
75  I will study and get ready  and perhaps my cha...   100            1   
76  I am not concerned that you fall; I am concern...   100            3   
77  I destroy my enemies when I make them my friends.   100            2   
78  If I had another face  do you think I would we...   100            2   
79  It is better to be silent and be thought a foo...   100            1   
80  Marriage is neither heaven nor hell  it is sim...   100            2   
81  The best way to get a bad law repealed is to e...   100            1   
82  And in the end  

In [192]:
k_preds.drop(['index'], axis=1, inplace=True)

In [197]:
k_preds['code'] = k_preds.code.map({1:1, 2:2, 3:3, 10:100, 20:100, 30:100})

In [212]:
print k_preds.groupby(['code', 'Predictions']).count()

                  quote  Predictions 3
code Predictions                      
1    1                3              3
     3                1              1
     100              1              1
2    2                6              6
3    3                2              2
100  1               16             16
     2               11             11
     3               34             34
     100             17             17


In [199]:
k_preds.code.value_counts()

100    78
2       6
1       5
3       2
Name: code, dtype: int64

In [186]:
zip(w_cvec_2.get_feature_names(),
    np.asarray(X_w_dtm2.sum(axis=0)).ravel())

[(u'00 00', 0),
 (u'00 000', 0),
 (u'00 100', 0),
 (u'00 12', 0),
 (u'00 1814', 0),
 (u'00 22', 0),
 (u'00 35', 0),
 (u'00 350', 0),
 (u'00 86', 0),
 (u'00 barrel', 0),
 (u'00 barrels', 0),
 (u'00 bath', 0),
 (u'00 bernie', 0),
 (u'00 cash', 0),
 (u'00 choice', 0),
 (u'00 chores', 0),
 (u'00 class', 0),
 (u'00 crockery', 0),
 (u'00 crops', 0),
 (u'00 dining', 0),
 (u'00 dozen', 0),
 (u'00 dry', 0),
 (u'00 friday', 0),
 (u'00 furs', 0),
 (u'00 haste', 0),
 (u'00 houses', 0),
 (u'00 included', 0),
 (u'00 know', 0),
 (u'00 left', 0),
 (u'00 lunch', 0),
 (u'00 pm', 0),
 (u'00 previous', 0),
 (u'00 railroad', 0),
 (u'00 rendering', 0),
 (u'00 saturday', 0),
 (u'00 smith', 0),
 (u'00 total', 0),
 (u'00 victorian', 0),
 (u'000 00', 0),
 (u'000 000', 0),
 (u'000 100', 0),
 (u'000 107', 0),
 (u'000 12', 0),
 (u'000 120', 0),
 (u'000 15', 0),
 (u'000 17', 0),
 (u'000 23d', 0),
 (u'000 26', 0),
 (u'000 50', 0),
 (u'000 60', 0),
 (u'000 abraham', 0),
 (u'000 accept', 0),
 (u'000 accidentally', 0),

In [202]:
k_preds[k_preds.code == 2]

quote  code  Predictions  \
37  one should never copy the method of an artist....     2            2   
38  The condition of perfection is idleness: the a...     2            2   
39  It is only shallow people who do not judge by ...     2            2   
40  A woman will flirt with anybody in the world a...     2            2   
41  To get into the best society nowadays one has ...     2            2   
42  It is very vulgar to talk about one's business...     2            2   

    Predictions 3  
37              2  
38              3  
39              3  
40              2  
41              2  
42              2

In [203]:
k_preds[k_preds.code == 1]

quote  code  Predictions  \
0  If you pick up a starving dog and make him pro...     1            1   
2  There has been only one Christian. They caught...     1            3   
3  The perfection of wisdom, and the end of true ...     1            1   
5  Part of my plan has been to try to pleasantly ...     1          100   
7  In the first place, God made idiots. That was ...     1            1   

   Predictions 3  
0              1  
2              2  
3              3  
5              1  
7              1

In [204]:
k_preds[k_preds.code ==3]

quote  code  Predictions  \
57  I am for those means which will give the great...     3            3   
58  Having thus chosen our course without guile an...     3            3   

    Predictions 3  
57              3  
58              1